In [ ]:
import autogen
from autogen.agentchat.contrib.retrieve_assistant_agent import RetrieveAssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent

In [ ]:
def termination_msg(msg):
        content = msg.get("content")
        if content.lower() in ("exit", "quit", "stop", "done", "terminate"):
            return True
        return False

In [ ]:
config_file="CONFIG_LIST.json"
gemini_filter_dict = {"model": ["gemini"]}
gemini_config_list = autogen.config_list_from_json(
    env_or_file=config_file, filter_dict=gemini_filter_dict
    )

In [ ]:
llm_config = {
    "functions": [
        {
            "name": "retrieve_content",
            "description": "retrieve content for code generation and question answering.",
            "parameters": {
                "type": "object",
                "properties": {
                    "message": {
                        "type": "string",
                        "description": "Refined message which keeps the original meaning and can be used to retrieve content for code generation and question answering.",
                    }
                },
                "required": ["message"],
            },
        },
    ],
    "config_list": gemini_config_list,
    "timeout": 120,
    "seed": 50,
}

In [ ]:
manager_llm_config = {
    "config_list": gemini_config_list,
    "timeout": 120,
    "seed": 50,
}

In [ ]:
programmer_prompt = ""
with open("prompts/Programmer.txt") as f:
    programmer_prompt = f.read()
programmer_prompt += """\nRemember, You are a senior python engineer.you will get product specification and will create appropriate code to develop the requiered product.
                        you should listen to the Code_Reviewer suggestions and improve you code if needed until there are no more fixes,
                        Code_Reviewer suggestions are marked under `SUGGESTION`. each of your response should include the full fixed code.
                        you also can use the retrieve_content function for extra matirial to solve your tasks,
                        the retrieve_content function have retrival power to relevent books libraries"""

In [ ]:
reviewer_prompt = ""
with open("prompts/Code_Reviewer.txt") as f:
    reviewer_prompt = f.read()
reviewer_prompt += """\n Remember, You are a code reviewer. you need to review code sections from the Senior_Python_Engineer that marked under `REVIEW` and provide suggestions and fixes.
                    mark your suggestions under `SUGGESTION`. Reply `DONE` when there are no more suggestions and fixes.
                    you also can use the retrieve_content function for extra matirial to solve your tasks,
                    the retrieve_content function have retrival power to relevent books libraries"""

In [ ]:
pm_prompt = ""
with open("prompts/PM.txt") as f:
    pm_prompt = f.read()
pm_prompt += """\nRemember, You are a product manager. you need to provide detailed full-stack web application description and specification.
                        When Code_Reviewer send message marked under `DONE`, Reply `TERMINATE` and end the conversation.
                        you should not replay on other messages.
                        you also can use the retrieve_content function for extra matirial to solve your tasks,
                        the retrieve_content function have retrival power to relevent books libraries"""

In [ ]:
ragassistant = RetrieveUserProxyAgent(
    name="Rag_Assistant",
    is_termination_msg=termination_msg,
    system_message="""Assistant who has extra content retrieval power for solving difficult problems.""",
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
    retrieve_config={
        "task": "qa",
        "docs_path": "rag/"
    },
)

In [ ]:
coder = RetrieveAssistantAgent(
    name="Senior_Python_Engineer",
    is_termination_msg=termination_msg,
    system_message=programmer_prompt,
    llm_config=llm_config,
)

In [ ]:
pm = RetrieveAssistantAgent(
    name="Product_Manager",
    is_termination_msg=termination_msg,
    system_message=pm_prompt,
    llm_config=llm_config,
)

In [ ]:
reviewer = RetrieveAssistantAgent(
    name="Code_Reviewer",
    is_termination_msg=termination_msg,
    system_message=reviewer_prompt,
    llm_config=llm_config,
)

In [ ]:
def retrieve_content(message, n_results=3):
        ragassistant.n_results = n_results  # Set the number of results to be retrieved.
        # Check if we need to update the context.
        update_context_case1, update_context_case2 = ragassistant._check_update_context(message)
        if (update_context_case1 or update_context_case2) and ragassistant.update_context:
            ragassistant.problem = message if not hasattr(ragassistant, "problem") else ragassistant.problem
            _, ret_msg = ragassistant._generate_retrieve_user_reply(message)
        else:
            ret_msg = ragassistant.generate_init_message(message, n_results=n_results)
        return ret_msg if ret_msg else message

for agent in [pm, coder, reviewer]:
    # register functions for all agents.
    agent.register_function(
        function_map={
            "retrieve_content": retrieve_content,
        }
    )

In [ ]:
groupchat = autogen.GroupChat(
    agents=[pm, coder, reviewer], messages=[], max_round=12,speaker_selection_method="round_robin"
)
coordinator = autogen.GroupChatManager(groupchat=groupchat, llm_config=manager_llm_config)

In [ ]:
# Start chatting with the boss as this is the user proxy agent.
coordinator.initiate_chat(
    coordinator,
    message="Create notification service, using python, which will load data from some db and will schedule nofitication base on the loaded records. the logic implementation is not defined so create only the abstractions for the class. eventually, give one example implementation for which will read from csv and schedule email notification base on the data metioned inside each record.",
)